# This project implemented by Elham Mahdipour
## I am a Ph.D. Graduate  of computer engineering at Yazd University, Yazd, Iran.
## I am a Faculty Member of Computer Engineering Department at Khavaran Institute of Higher Education, Mashhad, Iran.
### Please feel free and contact to me: elham.mahdipour@gmail.com/ e.mahdipour@profs.khi.ac.ir

# Please run all cells for test the RENA model. This program test RENA for RNorvegicus and CElegans species.

# Phase 1 
## Create Dataset and Preprocessing

In [1]:
import networkx as nx
G1=nx.read_leda('biogrid samples\RNorvegicus.gw')
G1

In [2]:
G2=nx.read_leda('biogrid samples\CElegans.gw')
G2

In [3]:
### Check and Swap if G1 > G2 ###
print(len(G1))
print(len(G2))
print('*******')
if len(G1)>len(G2):
    temp=G1
    G1=G2
    G2=temp
print(len(G1))
print(len(G2))

1657
3134
*******
1657
3134


In [4]:
G_target_na=nx.read_weighted_edgelist('biogrid samples\RN-CE.tsv')
print(len(G_target_na))
G_target_na  

5458


In [5]:
ed1=G1.edges()
ed2=G2.edges()

nd1=G1.nodes()
nd2=G2.nodes()

el1=list(ed1)
el2=list(ed2)

nd1=list(nd1)
nd2=list(nd2)

degG1 = [val for (node, val) in G1.degree()]
degG2 = [val for (node, val) in G2.degree()]

# Feature Extraction

# compute score for create similarity matrix

In [6]:
def deg_Diff(G1,G2):
    Degree_Difference=np.zeros((len(G1),len(G2)))
    for i in range(len(G1)):
        for j in range(len(G2)):
            Degree_Difference[i][j]=abs(degG1[i]-degG2[j])/max(degG1[i],degG2[j])
    return Degree_Difference

In [7]:
def score_pageRank(X):
    a=nx.pagerank(X)
    return a

In [8]:
def coefficient_pagerank(x,y):  #x is G1, y is G2
   # print(len(x))
    p1=score_pageRank(x)
    b=p1.values()
    pr1=list(b)
    p2=score_pageRank(y)
    c=p2.values()
    pr2=list(c)
    pr=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            #print(pr1[i],pr2[j])
            pr[i][j]=abs(pr1[i]-pr2[j])/max(pr1[i],pr2[j])   #minimum pr is maximum similarity of topology 
    return pr

In [9]:
def coefficient_edges(index_node, G, GraphNumber):
    if GraphNumber==1:
        sum_edge=0        
        for i in G.neighbors(nd1[index_node]):                       
            sum_edge=sum_edge+degG1[nd1.index(i)]
        #print(sum_edge)
        temp=(degG1[index_node]-1) if degG1[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG1[index_node]*temp)
    else:
        sum_edge=0        
        for i in G.neighbors(nd2[index_node]):                       
            sum_edge=sum_edge+degG2[nd2.index(i)]
        #print(sum_edge)
        temp=(degG2[index_node]-1) if degG2[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG2[index_node]*temp)
    return coeff_node

In [10]:
def compute_Ea(G1,G2):
    Ea_G1=np.zeros(len(G1))
    Ea_G2=np.zeros(len(G2))
    for i in range(len(G1)):
        Ea_G1[i]=coefficient_edges(i, G1, 1)
    for j in range(len(G2)):
        Ea_G2[j]=coefficient_edges(j,G2,2)
    ea=[Ea_G1, Ea_G2]
    return(ea)

In [11]:
# compute relative clustering coefficient difference between node a (in G1) and node b (in G2)
def CD(G1, G2):
    cd=np.zeros((len(G1),len(G2)))
    EA=compute_Ea(G1,G2)
    #print(EA[0])      #Ea for G1
    #print("===================")
    #print(EA[1])      #Ea for G2
    for i in range(len(G1)):
        for j in range(len(G2)):
            cd[i,j]=abs(EA[0][i]-EA[1][j])/max(EA[0][i],EA[1][j])
    return cd

In [12]:
elt=G_target_na.edges()
elt=list(elt)
print(len(elt))
elt

2729


[('252881', '173910'),
 ('314690', '190219'),
 ('305317', '176191'),
 ('50671', '172715'),
 ('363972', '180042'),
 ('314198', '189841'),
 ('297893', '173074'),
 ('499443', '180827'),
 ('60384', '177779'),
 ('81662', '171751'),
 ('170699', '173176'),
 ('363160', '175886'),
 ('297453', '183226'),
 ('680111', '172249'),
 ('25083', '184469'),
 ('619346', '180843'),
 ('316539', '173794'),
 ('25363', '174180'),
 ('80848', '179729'),
 ('302973', '178559'),
 ('89812', '175344'),
 ('685232', '3564970'),
 ('64201', '173414'),
 ('293098', '180740'),
 ('295247', '353452'),
 ('64205', '172943'),
 ('25675', '176047'),
 ('361925', '177325'),
 ('25718', '175410'),
 ('361239', '173418'),
 ('315707', '266817'),
 ('361251', '172890'),
 ('287830', '172611'),
 ('316611', '176490'),
 ('297708', '182937'),
 ('502421', '190665'),
 ('684558', '171696'),
 ('689914', '181571'),
 ('288414', '178259'),
 ('297911', '177679'),
 ('288908', '173244'),
 ('363195', '179677'),
 ('502414', '177845'),
 ('361626', '183942')

In [13]:
def sequence_score(x,y):
    seq=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            q1=(nd1[i],nd2[j]) in elt
            if q1==False:
                c=0
            else:
               # print(q1)
                c=1
            seq[i][j]=c    
            
    return seq

In [14]:
def compute_score(G1,G2):
    coeff_pr=coefficient_pagerank(G1,G2)
    dd=deg_Diff(G1,G2)
    cd=CD(G1,G2)
    seq_sc=sequence_score(G1,G2)
    
    alpha=0.1
    betta=0.2
    gamma=0.2
    zetta=1-alpha-betta-gamma
    s=alpha*(1-coeff_pr)+betta*(1-dd)+gamma*(1-cd)+zetta*seq_sc
    return s,coeff_pr, dd, cd, seq_sc

In [15]:
import numpy as np
m, coeff_pr, dd, cd, seq=compute_score(G1,G2)
sim=m

# Phase 2: Dataset Generation

# create data for deep learning

# change problem to classification 
## [node of G1, node of G2, sequence similarity, Coefficient page rank, clustering coefficient difference,  similarity score, alignment=yes(0)/TN or no(1)/TP]

In [16]:
en_mat=[]

for i in range(len(nd1)):
    for j in range(len(nd2)):
        if G_target_na.has_edge(nd1[i],nd2[j]):
            align_class='Yes'
        else:
            align_class='No'
        
        sample=[int(nd1[i]),int(nd2[j]),seq[i][j], coeff_pr[i][j], dd[i][j],cd[i][j],sim[i][j],align_class] 
        en_mat.append(sample)
print(len(en_mat))

5193038


In [17]:
yc=[]
noc=[]
for i in range(len(en_mat)):
    if en_mat[i][7]=='Yes': 
        yc.append(en_mat[i])
    else:
        noc.append(en_mat[i])
print(len(yc), len(noc))

228 5192810


In [18]:
data=yc+noc 
len(data)
X=[]
y=[]
for i in range(len(data)):
    X.append(data[i][0:7])
    y.append(data[i][7])
    
len(X),len(y)

(5193038, 5193038)

In [19]:
X_train=X
y_train=y

In [20]:
x_tr,y_tr=X,y

# test model for real data

In [21]:
# Label encode Class (Species)
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# One Hot Encode
y_train = np_utils.to_categorical(encoded_Y)

Using TensorFlow backend.


In [22]:
import numpy as np
X_train=np.array(X_train)
y_train=np.array(y_train)

# Phase 3: RENA network

## Load tune RENA model

In [29]:
from keras.models import load_model
#del model
model_rnn = load_model('biogrid samples/deep_model_undersample_RNN_SC-AT.h5')
model_rnn.load_weights('biogrid samples/deep_model_undersample_RNN_weights_SC-AT.h5', by_name=True)

C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [30]:
# without resample
result_tr = model_rnn.evaluate(X_train, y_train)
result_tr

5193038/5193038 [==============================] - 387s 75us/step

[0.001142512955390883, 1.0, 0.001141411237552715, 1.3034439501557626e-06]


In [31]:
%%time
out_tr=model_rnn.predict(X_train)
out_tr

Wall time: 2min 59s


array([[5.5663445e-06, 9.9999440e-01],
       [5.5663445e-06, 9.9999440e-01],
       [5.5663445e-06, 9.9999440e-01],
       ...,
       [9.9999642e-01, 3.5356209e-06],
       [9.9999642e-01, 3.5356209e-06],
       [9.9999642e-01, 3.5356209e-06]], dtype=float32)

In [32]:
# Create alignment file 
f = open("biogrid samples/align-RN-CE.txt", "w")
for i in range(len(out_tr)):
    if out_tr[i][1]>out_tr[i][0]:
        z=[X_train[i][0],X_train[i][1]]
        f.write(str(z))
f.close()

In [33]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_train.argmax(axis=1), out_tr.argmax(axis=1)))

[[5192810       0]
 [      0     228]]


In [34]:
from sklearn.metrics import f1_score, precision_score, recall_score
print(precision_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))
print(recall_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))
print(f1_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))

1.0
1.0
1.0


In [35]:
print(metrics.roc_auc_score(y_train, out_tr))
print(metrics.matthews_corrcoef(y_train.argmax(axis=1),out_tr.argmax(axis=1)))

1.0
1.0


# test other classifier without resample

In [36]:
x_tra,y_tra=X_train,y_train

In [37]:
X_train,y_train=x_tr,y_tr

In [38]:
%%time
# Linear Discriminant Analysis

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
print('Accuracy of LDA classifier on training set: {:.2f}'
     .format(lda.score(X_train, y_train)))

Accuracy of LDA classifier on training set: 1.00
Wall time: 16.1 s


In [39]:
%%time
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = lda.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

['No' 'No' 'Yes' ... 'No' 'No' 'No']
0.9999572504572468
[[5192810       0]
 [    222       6]]
0.9999786252039271
0.5131578947368421
0.5256303380145434
Wall time: 41.1 s


In [40]:
%%time
# KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))

Accuracy of KNN classifier on training set: 1.00
Wall time: 2min 33s


In [41]:
%%time
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = knn.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999560950641995
[[5192810       0]
 [    228       0]]


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.49997804753209973
0.5
0.49998902352508917
Wall time: 2min 43s


In [42]:
%%time
# SVM
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
print('Accuracy of SVM classifier on training set: {:.2f}'
     .format(svm.score(X_train, y_train)))

Accuracy of SVM classifier on training set: 1.00
Wall time: 1min 42s


In [43]:
%%time
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = svm.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999560950641995
[[5192810       0]
 [    228       0]]


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.49997804753209973
0.5
0.49998902352508917
Wall time: 1min 17s
